# Introduction

The provided notebook is a Python program for cryptocurrency data analysis and portfolio optimization. It uses historical price data and market cap data for selected cryptocurrencies to perform portfolio optimization using the Efficient Frontier approach. The script aims to calculate the profit/loss of the portfolio over different time periods and with varying numbers of coins.

# Notebook
## Import libs

In [1]:
import sys
sys.path.append("../")

import itertools
from time import time

import numpy as np
import pandas as pd


from src import cryptorama as cc

## Functions

In [2]:
def calculate_profit(df, _top_100, _n_coins, _mu_method, _cov_method, _obj_function, _budget, 
                     _n_days_vector, sell_day, _compounding, _save_dir):

    crypto_class = cc.CryptoPortfolio(top_100, _budget, _n_coins, _save_dir)
    results = {}
    portf = {}

    for n_days in _n_days_vector[:-1]:
        crypto_class.validate_from_past_specific_dates(_n_coins, float(n_days), sell_day, 
                                                       _mu_method, _cov_method, _obj_function, 
                                                       _compounding)
        date = df.iloc[n_days].name

        results[date] = crypto_class.p_l_specific
        portf[date] = crypto_class.portfolio_from_past_specific

    p_l = sum(results.values())

    return p_l, results, portf


def check_coins(portfolio):
    # Convert the dictionary to a list of DataFrames
    dfs = [pd.DataFrame(portfolio[date]) for date in portfolio]

    # Concatenate the list of DataFrames into a single DataFrame
    result_df = pd.concat(dfs, keys=portfolio.keys())

    def get_total_coins(data):
        print(data["Coin"].iloc[0], data["n_coins"].sum())

    result_df.groupby("Coin").apply(get_total_coins)

## Inputs

In [3]:
start = time()

top_100 = False


save_dir = './legacy_data'
buy_year = 4
sell_year = 2
sell_day = int(365 * sell_year)
n_days_vector = np.arange(int(365 * buy_year), int(365 * sell_year), -30)
df_mc = pd.read_csv('./legacy_data/All_cryptos_market_cap.csv', index_col=0)


## Optimization parameters

In [4]:
possible_mu_methods = ["mean", "exp", "capm"]
possible_cov_methods = ["sample", "exp"]
possible_obj_functions = ["quadratic", "sharpe", "min_volat"]
compounding = True
# Get all combinations of mu_method, cov_method, and obj_function
all_combinations = list(itertools.product(possible_mu_methods, possible_cov_methods, possible_obj_functions))

## Variables initialization

In [5]:
# you place 100e each month per category for the past n years
budget = 100

maxpl2 = 0
maxpl10 = 0
maxpl20 = 0


best_portf_2 = None
best_portf_10 = None
best_portf_20 = None

best_res_2 = None
best_res_10 = None 
best_res_20 = None  

optimized_variables = dict()

## Run Analysis

In [6]:
# Test the portfolio_optimization function with all combinations
start = time()
for mu_method, cov_method, obj_function in all_combinations:

    n_coins = 2
    print(f'\n===== {n_coins} COINS ======\n')
    
    try:
        p_l_2, results_2, portf_2 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                     obj_function, budget, n_days_vector, sell_day,
                                                     compounding, save_dir)
        
    except Exception as e:
        pass
    
    if p_l_2 >= maxpl2:
        maxpl2 = p_l_2
        best_portf_2 = portf_2
        best_res2 = results_2
        
        optimized_variables["2_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
        
    n_coins = 10
    print(f'\n===== {n_coins} COINS ======\n')
    
    try:
        p_l_10, results_10, portf_10 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                        obj_function, budget, n_days_vector, sell_day,
                                                        compounding, save_dir)
    except Exception as e:
        pass
    if p_l_10 >= maxpl10:
        maxpl10 = p_l_10
        best_portf_10 = portf_10
        best_res10 = results_10
        
        optimized_variables["10_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
            
    n_coins = 20

    print(f'\n===== {n_coins} COINS ======\n')

    try:
        p_l_20, results_20, portf_20 = calculate_profit(df_mc, top_100, n_coins, mu_method, 
                                                        cov_method, obj_function, budget, 
                                                        n_days_vector, sell_day,
                                                        compounding, save_dir)
    except Exception as e:
        pass
    
    if p_l_20 >= maxpl20:
        maxpl20 = p_l_20
        best_portf_20 = portf_20
        best_res20 = results_20
        
        optimized_variables["20_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
        
        

end = time()
print(f'Time elapsed: {(end - start) / 60}')


===== 2 COINS ======

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 988.2953683346965e

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 1125.668934049904e

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 1278.500032058188e

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 1178.020048786475e

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 1469.0796163284485e

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 1717.5161531031335e

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 1309.5779153404324e

Expected annual return: 207.9%
Annual volatility: 123.8%
Sharpe Ratio: 1.66
Invest 100.0

 Profit Loss: 73

Expected annual return: 46747.4%
Annual volatility: 2152.4%
Sharpe Ratio: 21.72
Invest 100.0

 Profit Loss: 32.0147284881218e

Expected annual return: 631.0%
Annual volatility: 138.6%
Sharpe Ratio: 4.54
Invest 100.0

 Profit Loss: 2.004357425074353e

Expected annual return: 1262.5%
Annual volatility: 178.8%
Sharpe Ratio: 7.05
Invest 100.0

 Profit Loss: -13.118298143399516e



/Users/dimitrisglenis/anaconda3/lib/python3.8/site-packages/pypfopt/risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


Expected annual return: 1262.5%
Annual volatility: 178.9%
Sharpe Ratio: 7.05
Invest 100.0

 Profit Loss: -49.80353463000769e

Expected annual return: 1262.5%
Annual volatility: 178.8%
Sharpe Ratio: 7.05
Invest 100.0

 Profit Loss: -28.522561102924076e


===== 2 COINS ======

Expected annual return: 168.5%
Annual volatility: 96.3%
Sharpe Ratio: 1.73
Invest 100.0

 Profit Loss: 740.6149473073307e

Expected annual return: 168.5%
Annual volatility: 96.3%
Sharpe Ratio: 1.73
Invest 100.0

 Profit Loss: 818.2490040200447e

Expected annual return: 168.5%
Annual volatility: 96.3%
Sharpe Ratio: 1.73
Invest 100.0

 Profit Loss: 951.158529148277e

Expected annual return: 168.5%
Annual volatility: 96.3%
Sharpe Ratio: 1.73
Invest 100.0

 Profit Loss: 865.838787197707e

Expected annual return: 168.5%
Annual volatility: 96.3%
Sharpe Ratio: 1.73
Invest 100.0

 Profit Loss: 1096.1060601777122e

Expected annual return: 168.5%
Annual volatility: 96.3%
Sharpe Ratio: 1.73
Invest 100.0

 Profit Loss: 1249.90

Expected annual return: 368.0%
Annual volatility: 73.6%
Sharpe Ratio: 4.97
Invest 100.0

 Profit Loss: 477.7449975016719e

Expected annual return: 368.0%
Annual volatility: 73.6%
Sharpe Ratio: 4.97
Invest 100.0

 Profit Loss: 434.2309348478254e

Expected annual return: 368.0%
Annual volatility: 73.6%
Sharpe Ratio: 4.97
Invest 100.0

 Profit Loss: 347.8199998332018e

Expected annual return: 6418.1%
Annual volatility: 289.8%
Sharpe Ratio: 22.14
Invest 99.99999999999999

 Profit Loss: 151.3040103035784e

Expected annual return: 6418.1%
Annual volatility: 289.8%
Sharpe Ratio: 22.14
Invest 99.99999999999999

 Profit Loss: -2.4577630715509065e

Expected annual return: 389.4%
Annual volatility: 74.5%
Sharpe Ratio: 5.20
Invest 100.0

 Profit Loss: -20.43148244992486e

Expected annual return: 680.1%
Annual volatility: 87.0%
Sharpe Ratio: 7.79
Invest 100.00000000000001

 Profit Loss: -30.195325688430923e



...

/Users/dimitrisglenis/anaconda3/lib/python3.8/site-packages/pypfopt/risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


Expected annual return: 162.9%
Annual volatility: 71.2%
Sharpe Ratio: 2.26
Invest 100.00000000000001

 Profit Loss: -27.765981678927616e

Expected annual return: 94.6%
Annual volatility: 50.0%
Sharpe Ratio: 1.85
Invest 99.99999999999999

 Profit Loss: -4.899826363072155e


===== 2 COINS ======

Expected annual return: 186.8%
Annual volatility: 116.3%
Sharpe Ratio: 1.59
Invest 100.0

 Profit Loss: 988.2953683346965e

Expected annual return: 186.8%
Annual volatility: 116.3%
Sharpe Ratio: 1.59
Invest 100.0

 Profit Loss: 1125.668934049904e

Expected annual return: 186.8%
Annual volatility: 116.3%
Sharpe Ratio: 1.59
Invest 100.0

 Profit Loss: 1278.500032058188e

Expected annual return: 186.8%
Annual volatility: 116.3%
Sharpe Ratio: 1.59
Invest 100.0

 Profit Loss: 1178.020048786475e

Expected annual return: 186.8%
Annual volatility: 116.3%
Sharpe Ratio: 1.59
Invest 100.0

 Profit Loss: 1469.0796163284485e

Expected annual return: 186.8%
Annual volatility: 116.3%
Sharpe Ratio: 1.59
Invest 

Expected annual return: 292.1%
Annual volatility: 125.3%
Sharpe Ratio: 2.32
Invest 100.0

 Profit Loss: 512.8808068674138e

Expected annual return: 292.1%
Annual volatility: 125.3%
Sharpe Ratio: 2.32
Invest 100.00000000000003

 Profit Loss: 498.14104894303233e

Expected annual return: 313.3%
Annual volatility: 125.0%
Sharpe Ratio: 2.49
Invest 100.0

 Profit Loss: 324.48478549994195e

Expected annual return: 15951.0%
Annual volatility: 1263.5%
Sharpe Ratio: 12.62
Invest 99.99999999999999

 Profit Loss: 64.85926181251556e

Expected annual return: 14309.6%
Annual volatility: 1196.9%
Sharpe Ratio: 11.95
Invest 100.00000000000001

 Profit Loss: -17.00283423266435e

Expected annual return: 323.6%
Annual volatility: 131.8%
Sharpe Ratio: 2.44
Invest 99.99999999999999

 Profit Loss: -21.99782984202174e

Expected annual return: 337.9%
Annual volatility: 125.5%
Sharpe Ratio: 2.68
Invest 100.0

 Profit Loss: -39.05867951387384e

Expected annual return: 582.0%
Annual volatility: 203.8%
Sharpe Ratio

Expected annual return: 301.3%
Annual volatility: 111.0%
Sharpe Ratio: 2.70
Invest 100.0

 Profit Loss: 1096.004729700156e

Expected annual return: 306.3%
Annual volatility: 110.8%
Sharpe Ratio: 2.75
Invest 100.0

 Profit Loss: 771.3725101509003e

Expected annual return: 299.5%
Annual volatility: 109.4%
Sharpe Ratio: 2.72
Invest 100.0

 Profit Loss: 1591.5952084442185e

Expected annual return: 299.5%
Annual volatility: 109.4%
Sharpe Ratio: 2.72
Invest 100.0

 Profit Loss: 1191.2380145757604e

Expected annual return: 271.0%
Annual volatility: 111.7%
Sharpe Ratio: 2.41
Invest 100.00000000000001

 Profit Loss: 953.8920629712771e

Expected annual return: 270.3%
Annual volatility: 110.9%
Sharpe Ratio: 2.42
Invest 100.0

 Profit Loss: 649.3819144402348e

Expected annual return: 267.8%
Annual volatility: 111.8%
Sharpe Ratio: 2.38
Invest 100.0

 Profit Loss: 474.15179603906284e

Expected annual return: 268.6%
Annual volatility: 114.2%
Sharpe Ratio: 2.34
Invest 100.0

 Profit Loss: 435.64242025

Expected annual return: 151.3%
Annual volatility: 77.0%
Sharpe Ratio: 1.94
Invest 100.0

 Profit Loss: 368.69359980887833e

Expected annual return: 151.9%
Annual volatility: 77.0%
Sharpe Ratio: 1.95
Invest 100.0

 Profit Loss: 357.82891718355756e

Expected annual return: 150.3%
Annual volatility: 74.8%
Sharpe Ratio: 1.98
Invest 100.0

 Profit Loss: 386.8464870251844e

Expected annual return: 153.3%
Annual volatility: 74.8%
Sharpe Ratio: 2.02
Invest 100.0

 Profit Loss: 322.2545959151329e

Expected annual return: 130.8%
Annual volatility: 74.8%
Sharpe Ratio: 1.72
Invest 100.0

 Profit Loss: 420.3416279330165e

Expected annual return: 136.2%
Annual volatility: 74.8%
Sharpe Ratio: 1.79
Invest 100.0

 Profit Loss: 441.1007763624257e

Expected annual return: 153.3%
Annual volatility: 74.8%
Sharpe Ratio: 2.02
Invest 100.0

 Profit Loss: 352.644123232796e

Expected annual return: 156.8%
Annual volatility: 74.8%
Sharpe Ratio: 2.07
Invest 100.0

 Profit Loss: 262.279394681013e

Expected annual 

In [7]:
check_coins(best_portf_20)
print(maxpl20)
optimized_variables["20_coins"]

AAVE 0.728346461750081
ADA 12836.70420616693
BNB 40.73186354600872
ETC 16.345834166160984
LINK 36.773991406711666
LUNC 3.709127275806955
MATIC 96.41354723740471
SOL 2.295635928215459
WBTC 0.006892854164734126
28876.42969893395


{'mu': 'exp', 's': 'exp', 'obj_function': 'sharpe'}

In [8]:
check_coins(best_portf_2)
print(maxpl2)
optimized_variables["2_coins"]

ETH 8.61494247643092
17400.01058668685


{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic'}

In [9]:
check_coins(best_portf_10)
print(maxpl10)
optimized_variables["10_coins"]

ADA 1884.2015481349235
BNB 85.48153971160352
UNI 0.6789613712772586
26828.904319823374


{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic'}

## Results

In [10]:
print(f'Profit 2c: {round(maxpl2, 2)} $')
print(f'Profit 10c: {round(maxpl10, 2)} $')
print(f'Profit 20c: {round(maxpl20, 2)} $')
# print(f'Profit 100c: {round(maxpl100, 2)} $')

print(f'\nInvestment: {3 * budget * len(n_days_vector)} $')
print(f'Final Profit: {round(maxpl2 + maxpl10 + maxpl20 , 2)} $') # + maxpl100
print(f'Final Profit: {round((maxpl2 + maxpl10 + maxpl20 ) / (3 * budget * len(n_days_vector)) * 100, 2)} %\n') # + maxpl100

end = time()
print(f'Time elapsed: {(end - start) / 60}')

Profit 2c: 17400.01 $
Profit 10c: 26828.9 $
Profit 20c: 28876.43 $

Investment: 7500 $
Final Profit: 73105.34 $
Final Profit: 974.74 %

Time elapsed: 4.870121304194132


# Conclusion

## 2 Coins

In [11]:
optimized_variables["2_coins"]

{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic'}

## 10 Coins

In [12]:
optimized_variables["10_coins"]

{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic'}

## 20 Coins

In [13]:
optimized_variables["20_coins"]

{'mu': 'exp', 's': 'exp', 'obj_function': 'sharpe'}